아이템 기반 필터링 구현하기

- 비회원 장르별 평점 순위 보여주기 기능 구현(장르별 네이버 평점 보여주기)
- 사용자 선호에 따른 장르, 감독별 유사도 검증 구현 

In [1]:
##########################################

# 비회원 장르별 순위 추천


In [2]:
# 1. 데이터 가져오기 
# 1-1. import 설정
import pymysql
import pandas as pd
import numpy as np


#1-2. DB에서 데이터 가져오기
# inner join을 이용해 movie, genre 테이블 가져오기

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
cursor = conn.cursor()

# sql 가져오기
sql = """
    select * from movie
"""

# sql 실행
cursor.execute(query=sql)
rows = cursor.fetchall()

# 결과 가져오기
# for row in rows:
#     print(row)

# 데이터프레임으로 변환하기
movie_db = pd.DataFrame(rows, columns=['movieCd', 'movieNm', 'movieNmEn', 'shortContent', 'prdtYear', 'showTm', 'openDt', 'naverScore', 'imdbScore', 'watchGradeCd'])
movie_db

,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None
1,19188001,개의 일생,A Dog's Life,None,1918,None,0,None,None,None
2,19190002,여름밤의 기묘한 이야기,Unheimliche Geschichten,None,1919,None,0,None,None,None
3,19190003,재판장,The President,None,1919,None,0,None,None,None
4,19190007,의리적 구토,Fight For Justice,None,1919,None,19191027,None,None,None
...,...,...,...,...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,From end to end,None,2021,None,0,None,None,None
46914,20219978,볼레로 만들기,Structure: Making Bolero,None,2020,None,0,None,None,None
46915,20219984,어디에도 없는 시간,Where would we go,None,2021,None,0,None,None,None
46916,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None


In [3]:
# 2. 유저 평점 데이터 가져오기
# 2-1. 유저 csv 파일 가져오기
userscore = pd.read_csv('./userscore.csv')

# 2-2. userscore 형태 변환 int -> str
userscore = userscore.astype(str)



In [4]:
userscore_matrix = userscore.pivot_table('userScore', 'userId', 'movieCd')
userscore_matrix

movieCd,19258004,19318011,19328007,19338003,19398001,19408011,19420001,19438008,19488007,19508001,...,20218333,20218391,20218604,20218706,20218904,20219115,20219260,20219331,20219725,20219892
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,1.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN
96,NaN,3.0,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# 3. userscore, movie_db 합치기
# 3-2. userscore, movie_db를 movieCd기준으로 합하기
movie_merge = pd.merge(movie_db, userscore, on='movieCd')

# 3-2. user평가의 중복값 제거하기
movie_merge = movie_merge.drop_duplicates(['movieCd','userId'])
movie_merge = pd.DataFrame(movie_merge)
movie_merge

# movie_merge.head()



,movieCd,movieNm,movieNmEn,shortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,userId,userScore
0,19258004,오페라의 유령,The Phantom of the Opera,가면 뒤에 흉측한 외모를 가리고 파리 오페라하우스의 지하에 숨어 사는 팬텀.\r\n...,1925,None,0,4.4,None,None,8,9
1,19258004,오페라의 유령,The Phantom of the Opera,가면 뒤에 흉측한 외모를 가리고 파리 오페라하우스의 지하에 숨어 사는 팬텀.\r\n...,1925,None,0,4.4,None,None,25,9
2,19258004,오페라의 유령,The Phantom of the Opera,가면 뒤에 흉측한 외모를 가리고 파리 오페라하우스의 지하에 숨어 사는 팬텀.\r\n...,1925,None,0,4.4,None,None,68,3
3,19258004,오페라의 유령,The Phantom of the Opera,가면 뒤에 흉측한 외모를 가리고 파리 오페라하우스의 지하에 숨어 사는 팬텀.\r\n...,1925,None,0,4.4,None,None,77,9
4,19318011,공공의 적,The Public Enemy,{나는 경찰이다. 대한민국 경찰은 모든 국민의 평안하고 행복한 삶에 대한 x나게 영...,1931,None,0,4.6,None,None,73,3
...,...,...,...,...,...,...,...,...,...,...,...,...
8419,20219725,걸어도 걸어도,Nice to meet you,"햇볕이 따갑던 어느 여름 날, 바다에 놀러 간 준페이는 물에 빠진 어린 소년 요시오...",2021,None,0,4.3,None,None,87,6
8420,20219892,스피릿,Spirit Untamed,"초록 풀빛이 파도를 치는 광활한 대지 위로 거릴 것 없이 질주하는 야생마들의 무리,...",2021,None,20210915,4.6,None,None,26,6
8421,20219892,스피릿,Spirit Untamed,"초록 풀빛이 파도를 치는 광활한 대지 위로 거릴 것 없이 질주하는 야생마들의 무리,...",2021,None,20210915,4.6,None,None,62,3
8422,20219892,스피릿,Spirit Untamed,"초록 풀빛이 파도를 치는 광활한 대지 위로 거릴 것 없이 질주하는 야생마들의 무리,...",2021,None,20210915,4.6,None,None,17,7


In [ ]:
####################################3

In [40]:
# 4. 피벗테이블 만들기
# 4-1. 피벗테이블 생성
score_matrix = movie_merge.pivot_table('userScore', 'userId', 'movieNm', )

# 4-2. NaN값 0으로 변환하기
score_matrix.fillna(0, inplace=True)

# 4-3. 아이템-사용자 평점 행렬로 전치하기
score_matrix.head(1000)

movieNm,...ing,100일 동안 100가지로 100퍼센트 행복찾기,10분,12 몽키즈,12번째 솔저,13층,1917,1919 유관순,1987,1급기밀,...,황혼의 사무라이,회복,후크,휘트니,흐르는 강물처럼,희극지왕,히노키오,히든 피겨스,히야,히트
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
12,0.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 5. 사용자 평가 영화의 유사도 산출

# 사용자 평가영화 유사도 행렬 설정
from sklearn.metrics.pairwise import cosine_similarity

movie_sim = cosine_similarity(score_matrix, score_matrix)

# 데이터 프레임 저장
movie_sim_df = pd.DataFrame(movie_sim, index=score_matrix.index, columns=score_matrix.index)
movie_sim_df.iloc[:20, :20]

# 
# len(movie_sim_df)

movieNm,...ing,100일 동안 100가지로 100퍼센트 행복찾기,10분,12 몽키즈,12번째 솔저,13층,1917,1919 유관순,1987,1급기밀,1번가의 기적,21 그램,3:10 투 유마,48미터,4등,4월 이야기,500일의 썸머,577 프로젝트,5일의 마중,69 식스티 나인
movieNm,,,,,,,,,,,,,,,,,,,,
...ing,1.000000,0.000000,0.096856,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.204503,0.000000,0.000000,0.000000,0.000000
100일 동안 100가지로 100퍼센트 행복찾기,0.000000,1.000000,0.258478,0.000000,0.000000,0.0,0.000000,0.583433,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.235819,0.000000,0.000000,0.000000,0.000000
10분,0.096856,0.258478,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.34286,0.000000,0.000000,0.403821,0.000000,0.000000,0.000000,0.000000
12 몽키즈,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.181415,0.000000,0.000000,0.000000,0.226122,0.0,0.00000,0.037113,0.435194,0.000000,0.290728,0.000000,0.272557,0.000000
12번째 솔저,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.199431,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.091505,0.000000,0.000000,0.000000
13층,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1917,0.000000,0.000000,0.000000,0.181415,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.028364,0.0,0.00000,0.000000,0.138954,0.000000,0.085091,0.000000,0.119659,0.000000
1919 유관순,0.000000,0.583433,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.089275,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.242560,0.000000
1987,0.000000,0.000000,0.000000,0.000000,0.199431,0.0,0.000000,0.000000,1.000000,0.000000,0.410941,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#####################

In [8]:
# 5-2. 특정 영화와 비슷한 유사도 10개
movie_sim_df["어벤져스"].sort_values(ascending=False)[1:10]

movieNm
툴리                 0.766054
아이스 에이지: 지구 대충돌    0.731343
천일의 스캔들            0.705346
선생 김봉두             0.686533
레터스 투 줄리엣          0.680824
맘마미아!2             0.673512
고                  0.638590
여배우는 오늘도           0.634811
두 교황               0.631975
Name: 어벤져스, dtype: float64

In [11]:
# 6. 개인 최적화 영화 추천하기

# 6-1. 최근접 이웃 협업 필터링 사용(개인 최적화 영화 추천)

def predict_score(score_arr, movie_sim_arr):
    sum_sr = score_arr @ movie_sim_arr
    sum_s_abs = np.array([np.abs(movie_sim_arr).sum(axis=1)])

    score_pred = sum_sr / sum_s_abs

    return score_pred


In [51]:
# 6-2. 사용자별 예측 평점 
score_pred = predict_score(score_matrix.values, movie_sim_df.values)

score_pred_matrix = pd.DataFrame(data=score_pred, index = score_matrix.index, 
                                 columns = score_matrix.columns)
score_pred_matrix.head(2)



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1644 is different from 100)